In [12]:
import re
import nltk.corpus
import spacy
from pprint import pprint
from twilio_app import all_sandwiches
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to C:\Users\Ivan
[nltk_data]     Korostenskij\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def clean_text(text):

    # Lowercase because we don't want to treat "Hello" and "hello" differently
    text = text.lower()

    # Remove unicode with regex because it's irrelevant for our task and might cause problems
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)

    # Remove stopwords
    """
    Stopwords are words that are so common that they don't carry much meaning.
    For example, "the", "a", "is", "are", etc.

    """
    stop = stopwords.words('english')
    text = " ".join([word for word in text.split() if word not in stop])

    return text


In [61]:
default_toppings = {
    "lettuce": True,
    "tomato": True,
    "onions": True,
    "pickles": True,
    "banana_peppers": False,
    "black_olives": False,
    "green_peppers": False,
    "jalapeno_peppers": False,
    "cucumbers": False,
    "spinach": False,
    "salt": False,
    "pepper": False,
    "oregano": False,
    "oil_vinegar": False,
    "mustard": False,
    "mayonnaise": True,
    "ranch": False,
    "chipotle_sauce": False,
    "honey_mustard": False,
    "spicy_mustard": False,
    "sub_dressing": False,
    "pepperhouse_gourmaise": False,
    "deli_sub_sauce": False,
    "vegan_ranch_dressing": False,
    "buttermilk_ranch": False,
}

nlp = spacy.load("en_core_web_sm")

In [64]:
def parse_sandwich_order(input_string):
    # Turn the input string into a spaCy document
    doc = nlp(input_string.lower())

    # Initialize the sandwich name and toppings dictionary
    sandwich_name = ""
    toppings = {}

    # Loop through the tokens in the input string
    for token in doc:
        # If the token is a noun or adjective that describes a sandwich, add it to the sandwich order
        if token.pos_ in ["NOUN", "PROPN", "ADJ"] and ("sub" in token.text or "sandwich" in token.text)\
                or token.text in [s_wish.lower() for s_wish in all_sandwiches]:
            sandwich_name += all_sandwiches[token.text] + " "

        # If the token is a negative phrase, remove the corresponding topping from the toppings dictionary
        elif token.dep_ == "neg" and token.head.pos_ == "NOUN" and token.head.text in [toppin.lower() for toppin in default_toppings.items()]:
            default_toppings[token.head.text] = False

        # If the token is a topping, add it to the toppings dictionary
        elif token.pos_ == "NOUN" and token.text.lower() in default_toppings:
            default_toppings[token.text] = True

        # most_likely_sandwich = []
        # sub_text_length = len(sandwich_name)
        # if sandwich_name != "":
        #     for sub in all_sandwiches:
        #         if sandwich_name in sub:
        #             most_likely_sandwich.append(len(sub)/sub_text_length)
        #     most_likely_sandwich = max(most_likely_sandwich)
        #     print(f"Most likely sandwich: {most_likely_sandwich}")

    return sandwich_name.strip(), default_toppings

In [63]:
# Test the function with a sample input
sub_order_text = "Let me get a hot Italian, no mayo"
print(sub_order_text)
# Clean the text
#sub_order_text_cleaned = clean_text(sub_order_text)

# Parse the sandwich order and get the sandwich name and toppings
sandwich_name, toppings = parse_sandwich_order(sub_order_text)
print("Sandwich order:")
pprint(sandwich_name)
print("Toppings:")
pprint(toppings)

Let me get a hot Italian, no mayo
Sandwich order:
''
Toppings:
{'banana_peppers': False,
 'black_olives': False,
 'buttermilk_ranch': False,
 'chipotle_sauce': False,
 'cucumbers': False,
 'deli_sub_sauce': False,
 'green_peppers': False,
 'honey_mustard': False,
 'jalapeno_peppers': False,
 'lettuce': True,
 'mayonnaise': True,
 'mustard': False,
 'oil_vinegar': False,
 'onions': True,
 'oregano': False,
 'pepper': False,
 'pepperhouse_gourmaise': False,
 'pickles': True,
 'ranch': False,
 'salt': False,
 'spicy_mustard': False,
 'spinach': False,
 'sub_dressing': False,
 'tomato': True,
 'vegan_ranch_dressing': False}
